In [1]:
%pylab inline
import numpy as np
import dace

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
nn = 84
weights = np.random.normal (size=(nn, nn)).astype('f')
lengths = np.random.randint(0, 255, (nn, nn)).astype('i')
buffer  = np.random.normal(size=(nn, lengths.max()+1)).astype('f')
ns = np.tile(np.r_[:nn],(nn,1))

In [3]:
def fwd(buffer):
    return np.sum(weights*buffer[ns,lengths], axis=1)

def vjp(g,buffer,gbuf=None):
    gb = np.tile(g[:,None],(1,nn)) # rev of sum(a, axis=1)
    gbuf = gbuf if isinstance(gbuf,np.ndarray) else np.zeros_like(buffer)
    gbuf[ns,lengths.T] += weights.T * gb # TODO check works for duplicates
    return gbuf

gbuf = vjp(np.ones(nn), buffer)
%timeit fwd(buffer)
%timeit vjp(np.ones(nn), buffer)

33.6 µs ± 51.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
93.9 µs ± 1.17 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


With the kernels defined in NumPy, let's try DaCe

In [4]:
fwd_dace = dace.program(fwd)

In [5]:
fwd_dace(buffer)

TypeError: Passing a numpy view (e.g., sub-array or "A.T") "__g_ns" to DaCe programs is not allowed in order to retain analyzability. Please make a copy with "numpy.copy(...)". If you know what you are doing, you can override this error in the configuration by setting compiler.allow_view_arguments to True.

In [ ]:
%timeit fwd_dace(buffer)